# Day 7: Gold price EDA

### Import and setup libraries

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

from plotly.subplots import make_subplots
import plotly.io as pio

pio.templates.default = "ggplot2" 

### Data imports and preprocessing

In [2]:
gold = pd.read_csv("../data/gold_prices.csv", parse_dates=["Date"], index_col="Date")

# Preprocess data to match with spot prices approximately
gold['Price'] = gold['Price'].astype(float)
gold['Price'] = gold['Price'] * 10.8
gold['Price'] = gold['Price'].round(0)

In [3]:
px.line(gold, x=gold.index, y="Price", title="Gold Prices (GLD) Over Time", labels={"Price": "Price (USD)"})

## Exploratory Data Analysis (EDA)

Let's dive into the gold price dataset to understand its structure, distribution, and key characteristics.


In [ ]:
# Dataset Overview
print("=" * 60)
print("DATASET OVERVIEW")
print("=" * 60)
print(f"Shape: {gold.shape}")v
print(f"\nDate Range: {gold.index.min()} to {gold.index.max()}")
print(f"Total Days: {len(gold)}")
print(f"\nData Types:\n{gold.dtypes}")
print(f"\nMissing Values:\n{gold.isnull().sum()}")


DATASET OVERVIEW
Shape: (2515, 1)

Date Range: 2016-01-11 00:00:00 to 2026-01-09 00:00:00
Total Days: 2515

Data Types:
Price    float64
dtype: object

Missing Values:
Price    0
dtype: int64


In [ ]:
# Statistical Summary
print("\n" + "=" * 60)
print("STATISTICAL SUMMARY")
print("=" * 60)
print(gold.describe())
print(f"\nSkewness: {gold['Price'].skew():.4f}")
print(f"Kurtosis: {gold['Price'].kurtosis():.4f}")



STATISTICAL SUMMARY
             Price
count  2515.000000
mean   1856.810736
std     662.427507
min    1113.000000
25%    1331.500000
50%    1778.000000
75%    1977.500000
max    4501.000000

Skewness: 1.7198
Kurtosis: 2.9658


In [ ]:
# Correlation with time-based features
gold['Year'] = gold.index.year
gold['Month'] = gold.index.month
gold['Quarter'] = gold.index.quarter
gold['DayOfWeek'] = gold.index.dayofweek

# Seasonal Analysis
monthly_avg = gold.groupby('Month')['Price'].mean()
quarterly_avg = gold.groupby('Quarter')['Price'].mean()

print("\n" + "=" * 60)
print("SEASONAL PATTERNS")
print("=" * 60)
print("\nAverage Price by Month:")
for month, price in monthly_avg.items():
    month_name = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                  'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'][month-1]
    print(f"  {month_name}: ${price:.2f}")

print("\nAverage Price by Quarter:")
quarter_names = ['Q1 (Jan-Mar)', 'Q2 (Apr-Jun)', 'Q3 (Jul-Sep)', 'Q4 (Oct-Dec)']
for quarter, price in quarterly_avg.items():
    print(f"  {quarter_names[quarter-1]}: ${price:.2f}")

# Year-over-year performance
yearly_avg = gold.groupby('Year')['Price'].agg(['mean', 'min', 'max'])
print("\nYearly Performance:")
print(yearly_avg.to_string())



SEASONAL PATTERNS

Average Price by Month:
  Jan: $1794.44
  Feb: $1730.72
  Mar: $1756.07
  Apr: $1831.43
  May: $1828.16
  Jun: $1823.17
  Jul: $1860.51
  Aug: $1862.27
  Sep: $1912.87
  Oct: $1953.84
  Nov: $1927.07
  Dec: $1991.02

Average Price by Quarter:
  Q1 (Jan-Mar): $1760.83
  Q2 (Apr-Jun): $1827.56
  Q3 (Jul-Sep): $1877.90
  Q4 (Oct-Dec): $1957.44

Yearly Performance:
             mean     min     max
Year                             
2016  1292.368421  1113.0  1410.0
2017  1292.828685  1193.0  1384.0
2018  1297.936255  1200.0  1391.0
2019  1420.376984  1295.0  1584.0
2020  1799.869565  1491.0  2094.0
2021  1817.777778  1701.0  1975.0
2022  1813.358566  1633.0  2068.0
2023  1948.872000  1818.0  2080.0
2024  2387.892857  1992.0  2781.0
2025  3433.456000  2626.0  4501.0
2026  4419.333333  4301.0  4474.0


## Gold-Themed Dashboard

A comprehensive visualization dashboard highlighting key patterns, distributions, and time-series behavior of gold prices.


In [ ]:
# Gold-themed color palette
gold_color_primary = "#FFD700"  # Gold
gold_color_dark = "#DAA520"     # Goldenrod
gold_color_light = "#FFF8DC"    # Cornsilk
bg_color = "#1a1a1a"            # Dark background
text_color = "#ffffff"          # White text

# Create comprehensive dashboard with subplots
fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=(
        "Price Trend Over Time",
        "Price Distribution",
        "Daily Returns Distribution",
        "Monthly Volatility",
        "Cumulative Returns",
        "Price Moving Averages"
    ),
    specs=[
        [{"secondary_y": False}, {"secondary_y": False}],
        [{"secondary_y": False}, {"secondary_y": False}],
        [{"secondary_y": False}, {"secondary_y": False}]
    ],
    vertical_spacing=0.12,
    horizontal_spacing=0.12
)

# 1. Price Trend
fig.add_trace(
    go.Scatter(
        x=gold.index,
        y=gold['Price'],
        mode='lines',
        name='Gold Price',
        line=dict(color=gold_color_primary, width=2),
        hovertemplate='<b>Date:</b> %{x}<br><b>Price:</b> $%{y:.2f}<extra></extra>'
    ),
    row=1, col=1
)

# 2. Price Distribution (Histogram)
fig.add_trace(
    go.Histogram(
        x=gold['Price'],
        name='Price Distribution',
        marker=dict(color=gold_color_dark),
        nbinsx=40,
        hovertemplate='<b>Price Range:</b> $%{x:.2f}<br><b>Frequency:</b> %{y}<extra></extra>'
    ),
    row=1, col=2
)

# 3. Daily Returns Distribution
fig.add_trace(
    go.Histogram(
        x=gold['Daily_Return'].dropna(),
        name='Daily Returns',
        marker=dict(color=gold_color_primary),
        nbinsx=50,
        hovertemplate='<b>Return:</b> %{x:.2f}%<br><b>Frequency:</b> %{y}<extra></extra>'
    ),
    row=2, col=1
)

# 4. Monthly Volatility
monthly_volatility = gold['Daily_Return'].resample('M').std()
fig.add_trace(
    go.Bar(
        x=monthly_volatility.index,
        y=monthly_volatility.values,
        name='Monthly Volatility',
        marker=dict(color=gold_color_dark),
        hovertemplate='<b>Month:</b> %{x|%B %Y}<br><b>Volatility:</b> %{y:.2f}%<extra></extra>'
    ),
    row=2, col=2
)

# 5. Cumulative Returns
cumulative_returns = ((1 + gold['Daily_Return'] / 100).cumprod() - 1) * 100
fig.add_trace(
    go.Scatter(
        x=gold.index,
        y=cumulative_returns,
        mode='lines',
        name='Cumulative Returns',
        line=dict(color=gold_color_primary, width=2),
        fill='tozeroy',
        fillcolor=f'rgba(255, 215, 0, 0.2)',
        hovertemplate='<b>Date:</b> %{x}<br><b>Return:</b> %{y:.2f}%<extra></extra>'
    ),
    row=3, col=1
)

# 6. Moving Averages
ma_50 = gold['Price'].rolling(window=50).mean()
ma_200 = gold['Price'].rolling(window=200).mean()

fig.add_trace(
    go.Scatter(
        x=gold.index,
        y=gold['Price'],
        mode='lines',
        name='Price',
        line=dict(color=gold_color_primary, width=1),
        opacity=0.7,
        hovertemplate='<b>Date:</b> %{x}<br><b>Price:</b> $%{y:.2f}<extra></extra>'
    ),
    row=3, col=2
)

fig.add_trace(
    go.Scatter(
        x=gold.index,
        y=ma_50,
        mode='lines',
        name='50-Day MA',
        line=dict(color=gold_color_dark, width=2, dash='dash'),
        hovertemplate='<b>Date:</b> %{x}<br><b>MA50:</b> $%{y:.2f}<extra></extra>'
    ),
    row=3, col=2
)

fig.add_trace(
    go.Scatter(
        x=gold.index,
        y=ma_200,
        mode='lines',
        name='200-Day MA',
        line=dict(color='#FFB6C1', width=2, dash='dot'),
        hovertemplate='<b>Date:</b> %{x}<br><b>MA200:</b> $%{y:.2f}<extra></extra>'
    ),
    row=3, col=2
)

# Update layout with gold theme
fig.update_layout(
    title={
        'text': '<b>Gold Price Analysis Dashboard</b><br><sub>Comprehensive Exploratory Data Analysis</sub>',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 24, 'color': gold_color_primary}
    },
    height=1400,
    showlegend=True,
    template='plotly_dark',
    hovermode='x unified',
    paper_bgcolor=bg_color,
    plot_bgcolor=bg_color,
    font=dict(color=text_color, size=11),
    legend=dict(
        orientation='v',
        yanchor='top',
        y=0.98,
        xanchor='right',
        x=0.99,
        bgcolor='rgba(0, 0, 0, 0.5)',
        bordercolor=gold_color_primary,
        borderwidth=1
    )
)

# Update axes
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='rgba(255, 215, 0, 0.1)')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='rgba(255, 215, 0, 0.1)')

# Update axis labels
fig.update_xaxes(title_text="Date", row=1, col=1)
fig.update_yaxes(title_text="Price (USD)", row=1, col=1)

fig.update_xaxes(title_text="Price (USD)", row=1, col=2)
fig.update_yaxes(title_text="Frequency", row=1, col=2)

fig.update_xaxes(title_text="Daily Return (%)", row=2, col=1)
fig.update_yaxes(title_text="Frequency", row=2, col=1)

fig.update_xaxes(title_text="Date", row=2, col=2)
fig.update_yaxes(title_text="Volatility (%)", row=2, col=2)

fig.update_xaxes(title_text="Date", row=3, col=1)
fig.update_yaxes(title_text="Cumulative Return (%)", row=3, col=1)

fig.update_xaxes(title_text="Date", row=3, col=2)
fig.update_yaxes(title_text="Price (USD)", row=3, col=2)

fig.show()


/tmp/ipykernel_107132/2029972619.py:66: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



In [ ]:
# Calculate daily returns for analysis
gold['Daily_Return'] = gold['Price'].pct_change() * 100
gold['Daily_Change'] = gold['Price'].diff()

print("\n" + "=" * 60)
print("DAILY RETURNS & CHANGES")
print("=" * 60)
print(f"Mean Daily Return: {gold['Daily_Return'].mean():.4f}%")
print(f"Std Dev Daily Return: {gold['Daily_Return'].std():.4f}%")
print(f"Min Daily Return: {gold['Daily_Return'].min():.4f}%")
print(f"Max Daily Return: {gold['Daily_Return'].max():.4f}%")



DAILY RETURNS & CHANGES
Mean Daily Return: 0.0591%
Std Dev Daily Return: 0.9333%
Min Daily Return: -6.4309%
Max Daily Return: 4.9345%


## Key Insights from EDA


In [ ]:
# Price Range and Volatility Analysis
price_range = gold['Price'].max() - gold['Price'].min()
price_range_pct = (price_range / gold['Price'].min()) * 100

print("=" * 60)
print("VOLATILITY & TREND INSIGHTS")
print("=" * 60)
print(f"Price Range: ${gold['Price'].min():.2f} - ${gold['Price'].max():.2f}")
print(f"Total Range: ${price_range:.2f} ({price_range_pct:.1f}% increase)")
print(f"Current Price: ${gold['Price'].iloc[-1]:.2f}")
print(f"\nVolatility Metrics:")
print(f"  Daily Volatility (σ): {gold['Daily_Return'].std():.4f}%")
print(f"  Annualized Volatility: {gold['Daily_Return'].std() * np.sqrt(252):.2f}%")

# Up vs Down days
up_days = (gold['Daily_Change'] > 0).sum()
down_days = (gold['Daily_Change'] < 0).sum()
total_trading_days = up_days + down_days

print(f"\nMarket Direction:")
print(f"  Up Days: {up_days} ({(up_days/total_trading_days)*100:.1f}%)")
print(f"  Down Days: {down_days} ({(down_days/total_trading_days)*100:.1f}%)")

# Winning and losing streaks
winning_days_pct = (gold['Daily_Return'] > 0).sum()
losing_days_pct = (gold['Daily_Return'] < 0).sum()

print(f"\nReturn Distribution:")
print(f"  Days with Positive Returns: {winning_days_pct} ({(winning_days_pct/len(gold))*100:.1f}%)")
print(f"  Days with Negative Returns: {losing_days_pct} ({(losing_days_pct/len(gold))*100:.1f}%)")


VOLATILITY & TREND INSIGHTS
Price Range: $1113.00 - $4501.00
Total Range: $3388.00 (304.4% increase)
Current Price: $4474.00

Volatility Metrics:
  Daily Volatility (σ): 0.9333%
  Annualized Volatility: 14.82%

Market Direction:
  Up Days: 1324 (54.2%)
  Down Days: 1117 (45.8%)

Return Distribution:
  Days with Positive Returns: 1324 (52.6%)
  Days with Negative Returns: 1117 (44.4%)
